<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# BigQuery API (python) - exercise solution

In this LAB notebook, you will try to use the [__BigQuery API__](https://googleapis.dev/python/bigquery/latest/index.html) for python.

You will need to perform the following tasks:

- create a new dataset
- create a table (ingest the CSV file `people-with-dups.txt`)
- query the table: extract the average salary for each gender
- delete the dataset


### Install the libraries

In [1]:
pip install -q google-cloud

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -q google-cloud-bigquery

Note: you may need to restart the kernel to use updated packages.


### Setup the environment

In [3]:
# Import the python libraries

import os
from google.cloud import bigquery

In [4]:
# Import the .json credentials and set the environment variable (TODO:student)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./credentials.json"

## Create a dataset

In [5]:
# Construct a BigQuery client object.

client = bigquery.Client()

__Note__: in order to avoid conflicts with your colleagues, please name the dataset_id by changing "LAB_EXERCISE" with "\<surname\>_LAB_EXERCISE"

In [6]:
# Set the "dataset_id" and the "location" of the data

dataset_id = "{}.LAB_EXERCISE".format(client.project)
location = "EU"

print("Dataset id: " + dataset_id)

Dataset id: preparazione-lezione-gcp.LAB_EXERCISE


In [7]:
# Construct a full Dataset object to send to the API.

dataset = bigquery.Dataset(dataset_id)
dataset.location = location

In [8]:
# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.

dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset preparazione-lezione-gcp.LAB_EXERCISE


## Create a BigQuery table by ingesting CSV data

__OPTION 1__: you can manually specify the table schema (see this [example](https://cloud.google.com/bigquery/docs/schemas#manually_specifying_schemas))

Hint: the `:` character is a non-standard separator, you have to manage it.

In [9]:
# Set "table_id_1" to the ID of the table to create

table_id_1 = "{}.{}.PEOPLE_1".format(client.project, dataset.dataset_id)
file_path = "./people-with-dups.txt"

print("Table id: " + table_id_1)

Table id: preparazione-lezione-gcp.LAB_EXERCISE.PEOPLE_1


In [10]:
# Define the BigQuery job
# DOCUMENTATION: https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.LoadJobConfig.html

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("firstName", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("middleName", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("lastName", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gender", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("birthDate", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("salary", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("ssn", "STRING", mode="NULLABLE"),
    ],
    skip_leading_rows=1,
    field_delimiter=':',  # the ':' character is a non-standard separator --> must be specified
    source_format=bigquery.SourceFormat.CSV, # The source file format (default CSV).
)

with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id_1, job_config=job_config)

job.result()  # Waits for the job to complete.

table = client.get_table(table_id_1)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id_1
    )
)

Loaded 103000 rows and 7 columns to preparazione-lezione-gcp.LAB_EXERCISE.PEOPLE_1


__OPTION 2__: you can automatically infer the schema using the auto-detection (see this [example](https://cloud.google.com/bigquery/docs/schema-detect#loading_data_using_schema_auto-detection))

Hint: the `:` character is a non-standard separator, you have to manage it.

In [11]:
# Set "table_id_2" to the ID of the table to create

table_id_2 = "{}.{}.PEOPLE_2".format(client.project, dataset.dataset_id)
file_path = "./people-with-dups.txt"

print("Table id: " + table_id_2)

Table id: preparazione-lezione-gcp.LAB_EXERCISE.PEOPLE_2


In [12]:
# Define the BigQuery job
# DOCUMENTATION: https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.LoadJobConfig.html

job_config = bigquery.LoadJobConfig(
    autodetect=True, # schema autodetection
    field_delimiter=':',  # the ':' character is a non-standard separator --> must be specified
    source_format=bigquery.SourceFormat.CSV, # The source file format (default CSV).
)

with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id_2, job_config=job_config)

job.result()  # Waits for the job to complete.

table = client.get_table(table_id_2)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id_2
    )
)

Loaded 103000 rows and 7 columns to preparazione-lezione-gcp.LAB_EXERCISE.PEOPLE_2


## Make a query

You will have to write a query which extracts the average salary for each gender.

In [13]:
# We will query both the tables, to show that the results are identical.

query_1 = """
    SELECT gender, AVG(salary)
    FROM `""" + table_id_1 + """`
    GROUP BY gender
"""

query_2 = """
    SELECT gender, AVG(salary)
    FROM `""" + table_id_2 + """`
    GROUP BY gender
"""

In [14]:
# Run the queries

query_job_1 = client.query(query_1)

query_job_2= client.query(query_2)

In [15]:
# Print the results

print("The query_1 data:")
for row in query_job_1:
    # Row values can be accessed by field name or index.
    print("gender={}, avg_salary={}".format(row[0], row[1]))
    
print("\nThe query_2 data:")
for row in query_job_2:
    # Row values can be accessed by field name or index.
    print("gender={}, avg_salary={}".format(row[0], row[1]))

The query_1 data:
gender=F, avg_salary=155504.28803229696
gender=M, avg_salary=155142.32038929235

The query_2 data:
gender=F, avg_salary=155504.28803229696
gender=M, avg_salary=155142.32038929235


## Clean up

In [16]:
# Delete the dataset

client.delete_dataset(
    dataset_id, delete_contents=True, not_found_ok=True
)

print("Deleted dataset '{}'.".format(dataset_id))

Deleted dataset 'preparazione-lezione-gcp.LAB_EXERCISE'.


##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2021 Quantia Consulting, srl. All rights reserved.